# Preprocess data using RoBERTa

In [72]:
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import tqdm
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
import scipy.sparse
import scipy.io as sio
from scipy.sparse import csc_matrix
import heapq
import pickle
#import tensorflow_hub as hub
import warnings
import random
from collections import defaultdict
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax

SOURCE_PATH="C:\\Users\\14106\\GAT\\100school\\"
YEAR='2020'
DEST_PATH="C:\\Users\\14106\\GAT\\preprocessed"+YEAR+"\\"
ORIG_UNIV=['columbia',"notredame","uofm","UCSD",'berkeley',"Harvard","ucla", 'dartmouth']
#ORIG_UNIV=["UCSD"]

'''
LINK = f"cardiffnlp/twitter-roberta-base-sentiment"
TOKENIZER = AutoTokenizer.from_pretrained(LINK)
CONFIG = AutoConfig.from_pretrained(LINK)
MODEL = AutoModelForSequenceClassification.from_pretrained(LINK)
ATTRIBUTE=["id","body"]
'''

'\nLINK = f"cardiffnlp/twitter-roberta-base-sentiment"\nTOKENIZER = AutoTokenizer.from_pretrained(LINK)\nCONFIG = AutoConfig.from_pretrained(LINK)\nMODEL = AutoModelForSequenceClassification.from_pretrained(LINK)\nATTRIBUTE=["id","body"]\n'

In [73]:
def preprocess(text):
    ''' Preprocess text (username and link placeholders)'''
    if not isinstance(text, str):
        return ''
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def get_files_name(year: str):
    files = os.listdir(SOURCE_PATH)
    files = [f for f in files if f[-8:-4]==year]
    return files
def get_subreddits():
    os.chdir(SOURCE_PATH)
    dat = pd.read_csv('colleges-CCIHE-final.csv')
    schools = []
    for i in range(len(dat.name)):
        schools.append(dat.loc[i, 'subreddit'])
    return schools

def get_feature(text:str):
    encoded_input = TOKENIZER(text, return_tensors='pt',max_length=512,truncation=True)
    output = MODEL(**encoded_input,output_hidden_states=True)
    scores = output[0][0].detach().numpy()
    scores = [round(float(i),4) for i in softmax(scores)]
    hidden_states=output[-1]#[1][0][0].shape
    emb_torch=hidden_states[-1][0][0]
    emb = emb_torch.cpu().detach().numpy()
    return scores, emb


In [ ]:
#continue previous block
#preprocess using roBERT, turn comment and submission combined data to feature, and roberta score
files=get_files_name(YEAR)
files=['comment_'+i+'_'+YEAR+'.csv' for i in ORIG_UNIV]
for f in tqdm.tqdm(['comment_'+'columbia'+'_'+YEAR+'.csv']):#(files):
    os.chdir(SOURCE_PATH)
    data=pd.read_csv(f,skip_blank_lines=True)
    data=data[ATTRIBUTE]
    data['emo_pred_pos']=0
    data['emo_pred_neu']=0
    data['emo_pred_neg']=0

    for d in range(0,len(data['id']),1):
        text = preprocess(data["body"][d])
        #text=text.encode('utf-8').decode('unicode-escape', errors='surrogatepass')
        scores, emb = get_feature(text)
        data.loc[d,['emo_pred_neg', 'emo_pred_neu', 'emo_pred_pos']]=scores
        os.chdir(DEST_PATH)
        name="feature_"+f[8:-9]+'.csv'
        e=open(name, 'a',newline='')
        with e:
            writer = csv.writer(e, delimiter=',')
            writer.writerow(emb)
    name=f[8:]
    os.chdir(DEST_PATH)
    data.to_csv(name, sep=',',mode='a',header=True, encoding="utf-8-sig")

In [5]:
get_files_name(YEAR)[116:]

['comment_UTAustin_2021.csv',
 'comment_UVA_2021.csv',
 'comment_UWMadison_2021.csv',
 'comment_Vanderbilt_2021.csv',
 'comment_villanova_2021.csv',
 'comment_VirginiaTech_2021.csv',
 'comment_washu_2021.csv',
 'comment_WellesleyCollege_2021.csv',
 'comment_wfu_2021.csv',
 'comment_williamandmary_2021.csv',
 'comment_WMU_2021.csv',
 'comment_WVU_2021.csv',
 'comment_yale_2021.csv']

# Adjacency matrix

In [47]:
from scipy import sparse

def base36encode(number, alphabet='0123456789abcdefghijklmnopqrstuvwxyz'):
    """Converts an integer to a base36 string."""
    if math.isnan(number):
        return ''
    number=int(number)
    if not isinstance(number, int):
        return ''
 
    base36 = ''
    sign = ''
 
    if number < 0:
        sign = '-'
        number = -number
 
    if 0 <= number < len(alphabet):
        return sign + alphabet[number]
 
    while number != 0:
        number, i = divmod(number, len(alphabet))
        base36 = alphabet[i] + base36
 
    return sign + base36
 
def base36decode(number):
    return int(number, 36)
 

In [23]:
#form adjacency matrix 
files=get_files_name(YEAR)
files=['comment_'+i+'_'+YEAR+'.csv' for i in ORIG_UNIV]
for f in tqdm.tqdm(files):
    os.chdir(SOURCE_PATH)
    data=pd.read_csv(f,skip_blank_lines=True)
    dic_c={}
    for d in range(len(data['id'])):
        id_c=data['id'][d][1:-1]
        dic_c[id_c]=d
    adj_cc=csc_matrix((d+1,d+1), dtype=np.int8)
    adj_acc=csc_matrix((d+1,d+1), dtype=np.int8)
    for d in range(len(data['id'])):
        id_c=data['id'][d][1:-1]
        #parent=base36encode(data['parent_id'][d])
        #print(parent)
        
        try:
            parent=data['parent_id'][d][4:-1]
        except:
            parent=''
        
        if parent in dic_c:
            adj_cc[dic_c[id_c],dic_c[parent]]=1
            adj_acc[dic_c[id_c],dic_c[parent]]=1
            adj_cc[dic_c[parent],dic_c[id_c]]=1
    #print(adj_cc.sum())
    print(adj_cc.sum())
    print(adj_acc.sum())
    os.chdir(DEST_PATH)
    name="dic_c_"+f[8:-9]+".p"
    pickle.dump(dic_c,open(name,"wb"))
    name="CCsym_"+f[8:-9]+".npz"
    sparse.save_npz(name, adj_cc)
    name="CCasym_"+f[8:-9]+".npz"
    sparse.save_npz(name, adj_acc)


  0%|                                                                                 | 0/8 [00:00<?, ?it/s]c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\scipy\sparse\_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
 12%|█████████▏                                                               | 1/8 [00:08<01:00,  8.60s/it]

6976
3488


c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\scipy\sparse\_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
 25%|██████████████████▎                                                      | 2/8 [00:16<00:50,  8.33s/it]

7678
3839


c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\scipy\sparse\_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
 38%|███████████████████████████▍                                             | 3/8 [00:27<00:46,  9.31s/it]

...

2952
1476


c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\scipy\sparse\_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
 88%|███████████████████████████████████████████████████████████████▉         | 7/8 [00:56<00:07,  7.54s/it]

7598
3799


c:\users\14106\appdata\local\programs\python\python37\lib\site-packages\scipy\sparse\_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
100%|█████████████████████████████████████████████████████████████████████████| 8/8 [00:57<00:00,  7.21s/it]

730
365


In [19]:
data['parent_id']#[d]#[4:-1]
dic_c
parent
id_c
adj_cc.sum()


730

In [24]:

print(adj_cc.sum())
print(adj_acc.sum())
os.chdir('C:\\Users\\14106\\GAT\\100school\\columbia\\')
name="CCsym_columbia2019.npz"
dat_cc=scipy.sparse.load_npz(name);
print(dat_cc.sum())# 6119 asym
adj_cc
#dat_cc
DEST_PATH

730
365
1993


'C:\\Users\\14106\\GAT\\preprocessed2020\\'

# Split large school dataset

In [74]:
NUM_SAMPLE = 30000 # if size of one school dataset > CUTOFF< then split it into 
INITIAL_SAMPLE = 50
ADJ='CCasym'

def split_graph(n):
    '''
    n is total number of instance in dataset
    output list of cutoff indices according to CUTOFF
    '''
    if n > CUTOFF:
        num_splits=n//CUTOFF+int(n%CUTOFF!=0)
        indices=[0]
        step=n//num_splits
        for i in range(num_splits-1):
            indices.append(step*(i+1))
        indices.append(n)
        return indices
    else:
        return [0,n]
    
def get_school_name():
    files = os.listdir(DEST_PATH)
    files = [f[8:-4] for f in files if f[0:7]=='feature']
    return files

def get_subreddits():
    os.chdir(SOURCE_PATH)
    dat = pd.read_csv('colleges-CCIHE-final.csv')
    schools = []
    for i in range(len(dat.name)):
        schools.append(dat.loc[i, 'subreddit'])
    return schools

def get_adj_martix_list():
    files = os.listdir(DEST_PATH)
    files = [f for f in files if f[0:len(ADJ)]==ADJ]
    return files

def sample_index(s:str):
    '''
    s: school name
    output: sampled indices
    '''
    os.chdir(DEST_PATH)
    name=ADJ+"_"+s+".npz"
    dat_cc=scipy.sparse.load_npz(name);
    n=dat_cc.shape[0]
    if n<=NUM_SAMPLE:
        selected=[i for i in range(n)]
        name='idx_'+s+'.csv'
        np.savetxt(name, selected, delimiter =", ")
        return selected
    #transform matrix into adjacency list
    d = dat_cc.todok()
    dic=defaultdict(list)
    for e in d.keys():
        dic[e[0]].append(e[1])

    selected=set() # nodes that both itself and its child has been traversed
    unselected=set([i for i in range(0,n)]) #  nodes that haven't traversed
    q=list(random.sample(tuple(unselected), INITIAL_SAMPLE)) # nodes that only itself has been traversed, and its children is waiting to be traversed
    unselected=unselected-set(q)
    while len(selected)<NUM_SAMPLE:
        cur=q.pop(0)
        samples_candi=dic[cur]
        for instance in samples_candi:
            if instance in unselected:
                q.append(instance)#
                unselected.remove(instance)
        selected.add(cur)
        if len(q)==0:
            q=list(random.sample(tuple(unselected), INITIAL_SAMPLE))
            unselected=unselected-set(q)
    selected=list(selected)
    selected.sort()
    name='idx_'+s+'.csv'
    np.savetxt(name, selected, delimiter =", ")
    return selected

def sample_feature(s:str, idx:list):
    os.chdir(DEST_PATH)
    name="feature_"+s+".csv"
    truefeatures=pd.read_csv(name,skip_blank_lines=True,header=None).values
    truefeatures=pd.DataFrame(truefeatures[idx,:])
    name="feature_sampled_"+s+'.csv'
    truefeatures.to_csv(name, header=False,index=False)
    
def sample_adj_matrix(s:str, idx:list):
    os.chdir(DEST_PATH)
    name=ADJ+"_"+s+".npz"
    dat_cc=scipy.sparse.load_npz(name);
    new_dat=dat_cc[idx,:][:,idx]
    name=ADJ+"_sampled_"+s+".npz"
    sparse.save_npz(name, new_dat)
    
def sample_bert_result(s:str, idx:list):
    os.chdir(DEST_PATH)
    name=s+'_'+YEAR+".csv"
    truefeatures=pd.read_csv(name,skip_blank_lines=True)
    truefeatures=truefeatures.iloc[idx,:]
    name=s+'_'+YEAR+'_sampled.csv'
    truefeatures.to_csv(name, header=True,index=False, encoding="utf-8-sig")

In [75]:
schools=get_subreddits()
#schools=ORIG_UNIV
random.seed(10)
for s in tqdm.tqdm(schools):
    idx=sample_index(s)
    #name='idx_'+s+'.csv'
    #idx=pd.read_csv(name,skip_blank_lines=True,header=None).values
    #idx=[int(i) for i in idx]
    sample_adj_matrix(s, idx)
    sample_feature(s, idx)
    sample_bert_result(s, idx)

100%|█████████████████████████████████████████████████████████████████████| 128/128 [58:13<00:00, 27.29s/it]


In [20]:
s

'columbia'

# draft

In [41]:

#print(adj_cc.sum())
#print(adj_acc.sum())
os.chdir('C:\\Users\\14106\\GAT\\preprocessed2019\\')
name="CCsym_sampled_columbia.npz"
dat_cc=scipy.sparse.load_npz(name);
print(dat_cc.sum())# 6119 asym
print(dat_cc[3,:].sum())
dat_cc


12238
6


<5031x5031 sparse matrix of type '<class 'numpy.int8'>'
	with 12238 stored elements in Compressed Sparse Column format>

In [ ]:
print(dic)

In [13]:
files=get_files_name(YEAR)
for f in tqdm.tqdm(files):
    os.chdir(SOURCE_PATH)
    data=pd.read_csv(f,skip_blank_lines=True)
    data=data[ATTRIBUTE]
    if len(data['id'])>CUTOFF:
        os.chdir(DEST_PATH)
        
        myFile = open('feature_berkeley.csv')
        print("The content of CSV file is:")
        lines = csv.reader(myFile)
        for line in lines:
            print(line)
            break
        myFile.close()

[0, 20230]

In [ ]:
# Draft below

In [24]:
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import tqdm
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
from scipy.sparse import csc_matrix
from scipy import sparse
import heapq
import pickle
#import tensorflow_hub as hub
import warnings
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import pickle
import scipy.sparse
from scipy.sparse import csc_matrix
from scipy import sparse

os.chdir('D:\\NLP\\sentiment2\\')

name="CCasym_notredame2021.npz"
dat_cc=scipy.sparse.load_npz(name);
dat=dat_cc.transpose(axes=None, copy=True)

In [25]:
(dat_cc!=dat).sum()

6982

In [4]:
f

'c'